# Hosted Agents on Microsoft Foundry

Deploy a **Hosted Agent** to Microsoft Foundry using the Microsoft Agent Framework.

This lab deploys a **Mars Exploration Expert** agent that answers questions about Mars missions.

> ⚠️ **Region**: Hosted Agents require **North Central US**

## Step 1: Configuration

In [ ]:
import subprocess
import os
import time
import json
import requests

# Unique suffix for all resources
SUFFIX = subprocess.run('az ad signed-in-user show --query id -o tsv', shell=True, capture_output=True, text=True).stdout.strip()[:6]

# Configuration
RESOURCE_GROUP = f"foundry-mars-{SUFFIX}"
LOCATION = "northcentralus"
FOUNDRY_NAME = f"mars-{SUFFIX}"
PROJECT_NAME = "mars-project"
ACR_NAME = f"acrmars{SUFFIX}"
AGENT_NAME = f"mars-expert-{int(time.time()) % 10000}"

print(f"Resource Group:  {RESOURCE_GROUP}")
print(f"Foundry Name:    {FOUNDRY_NAME}")
print(f"ACR Name:        {ACR_NAME}")
print(f"Agent Name:      {AGENT_NAME}")

In [ ]:
# Load APIM credentials
env_file = '/workspaces/getting-started-with-foundry/.env'
with open(env_file) as f:
    for line in f:
        if line.strip() and not line.startswith('#') and '=' in line:
            key, value = line.strip().split('=', 1)
            os.environ[key] = value

APIM_URL = os.environ['APIM_URL']
APIM_KEY = os.environ['APIM_KEY']
MODEL_NAME = os.environ['MODEL_NAME']

print(f"APIM URL:    {APIM_URL}")
print(f"Model:       {MODEL_NAME}")

## Step 2: Create Agent Code

In [3]:
# Create agent directory
AGENT_DIR = os.path.join(os.getcwd(), "mars-agent")
os.makedirs(AGENT_DIR, exist_ok=True)

# main.py
main_py = f'''
import os
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from azure.ai.agentserver.agentframework import from_agent_framework

def main():
    apim_url = os.getenv("APIM_URL")
    apim_key = os.getenv("APIM_KEY")
    model_name = os.getenv("MODEL_NAME", "gpt-4.1-mini")
    
    # APIM needs deployment path in base_url
    base_url = f"{{apim_url}}/deployments/{{model_name}}"
    
    chat_client = OpenAIChatClient(
        model_id=model_name,
        base_url=base_url,
        api_key=apim_key,
        default_headers={{"api-key": apim_key}},
    )
    
    agent = ChatAgent(
        chat_client,
        name="{AGENT_NAME}",
        id="{AGENT_NAME}",
        instructions="""
        You are an expert on Mars exploration and NASA\'s Mars missions. You have deep
        knowledge of rovers (Curiosity, Perseverance, Opportunity, Spirit), Mars landers,
        orbiters, and the search for life on Mars.
        
        Key missions you know about:
        - Mars 2020 / Perseverance: Sample collection, Ingenuity helicopter
        - Mars Science Laboratory / Curiosity: Exploring Gale Crater since 2012
        - InSight: Studying Mars interior and seismology
        - Mars Reconnaissance Orbiter: High-resolution imaging
        
        Keep responses informative but concise.
        """
    )
    
    from_agent_framework(agent).run()

if __name__ == "__main__":
    main()
'''

# requirements.txt
requirements = '''azure-ai-agentserver-agentframework==1.0.0b5
openai>=1.0.0
azure-identity>=1.15.0'''

# Dockerfile
dockerfile = '''FROM python:3.12-slim
WORKDIR /app
RUN apt-get update && apt-get install -y curl ca-certificates && rm -rf /var/lib/apt/lists/*
COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt
COPY main.py ./
EXPOSE 8088
CMD ["python", "main.py"]'''

for name, content in [("main.py", main_py), ("requirements.txt", requirements), ("Dockerfile", dockerfile)]:
    with open(os.path.join(AGENT_DIR, name), "w") as f:
        f.write(content.strip())

print(f"✅ Agent files created in {AGENT_DIR}")
!ls -la {AGENT_DIR}

✅ Agent files created in /workspaces/getting-started-with-foundry/04-hosted-agents/mars-agent
total 12
drwxr-xr-x 5 vscode vscode  160 Jan 21 14:45 .
drwxr-xr-x 7 vscode vscode  224 Jan 21 14:45 ..
-rw-r--r-- 1 vscode vscode  257 Jan 21 14:45 Dockerfile
-rw-r--r-- 1 vscode vscode 1404 Jan 21 14:45 main.py
-rw-r--r-- 1 vscode vscode   81 Jan 21 14:45 requirements.txt


## Step 3: Deploy Infrastructure

In [ ]:
# Create resource group
!az group create -n "{RESOURCE_GROUP}" -l "{LOCATION}" -o table

In [ ]:
# Get principal ID for RBAC
PRINCIPAL_ID = subprocess.run('az ad signed-in-user show --query id -o tsv', shell=True, capture_output=True, text=True).stdout.strip()

print(f"Deploying infrastructure (3-5 min)...")
!az deployment group create -g "{RESOURCE_GROUP}" --template-file main.bicep \
    -p foundryAccountName="{FOUNDRY_NAME}" \
    -p projectName="{PROJECT_NAME}" \
    -p acrName="{ACR_NAME}" \
    -p deployerPrincipalId="{PRINCIPAL_ID}" \
    -p location="{LOCATION}" \
    -o table

In [ ]:
# Get deployment outputs
result = subprocess.run(
    f'az deployment group show -g "{RESOURCE_GROUP}" -n main --query properties.outputs -o json',
    shell=True, capture_output=True, text=True
)
outputs = json.loads(result.stdout)

ACCOUNT_NAME = outputs['accountName']['value']
PROJECT_ENDPOINT = outputs['projectEndpoint']['value']
ACR_LOGIN_SERVER = outputs['acrLoginServer']['value']

print(f"Account:     {ACCOUNT_NAME}")
print(f"Project:     {PROJECT_ENDPOINT}")
print(f"ACR:         {ACR_LOGIN_SERVER}")

## Step 4: Create Capability Host

In [7]:
# Get credentials for REST API
SUBSCRIPTION_ID = subprocess.run('az account show --query id -o tsv', shell=True, capture_output=True, text=True).stdout.strip()
ACCESS_TOKEN = subprocess.run('az account get-access-token --resource https://management.azure.com/ --query accessToken -o tsv', shell=True, capture_output=True, text=True).stdout.strip()

headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ACCESS_TOKEN}"}

# Create Capability Host with enablePublicHostingEnvironment
capability_host_url = f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{ACCOUNT_NAME}/capabilityHosts/agents?api-version=2025-10-01-preview"

payload = {"properties": {"capabilityHostKind": "Agents", "enablePublicHostingEnvironment": True}}

print("Creating Capability Host...")
response = requests.put(capability_host_url, headers=headers, json=payload)
print(f"Status: {response.status_code}")

if response.status_code not in [200, 201, 409]:
    print(response.text)

Creating Capability Host...
Status: 201


In [8]:
# Wait for Capability Host to be ready
print("Waiting for Capability Host (2-5 min)...")
for i in range(30):
    response = requests.get(capability_host_url, headers=headers)
    if response.status_code == 200:
        state = response.json().get('properties', {}).get('provisioningState', 'Unknown')
        print(f"  [{i*10}s] {state}")
        if state == 'Succeeded':
            print("\n✅ Capability Host ready!")
            break
        elif state == 'Failed':
            print("\n❌ Failed")
            break
    time.sleep(10)
else:
    print("\n⚠️ Timeout")

Waiting for Capability Host (2-5 min)...
  [0s] Creating
  [10s] Creating
  [20s] Creating
  [30s] Creating
  [40s] Creating
  [50s] Creating
  [60s] Creating
  [70s] Creating
  [80s] Creating
  [90s] Creating
  [100s] Creating
  [110s] Creating
  [120s] Creating
  [130s] Creating
  [140s] Creating
  [150s] Creating
  [160s] Creating
  [170s] Creating
  [180s] Creating
  [190s] Creating
  [200s] Creating
  [210s] Creating
  [220s] Creating
  [230s] Creating
  [240s] Creating
  [250s] Creating
  [260s] Creating
  [270s] Creating
  [280s] Succeeded

✅ Capability Host ready!


## Step 5: Build & Push Container

In [ ]:
# Build and push to ACR (must use linux/amd64 platform)
print(f"Building {AGENT_NAME}:latest for linux/amd64...")
!az acr build --registry "{ACR_NAME}" --image "{AGENT_NAME}:latest" --platform linux/amd64 ./mars-agent/

## Step 6: Register Agent

In [10]:
!pip install azure-ai-projects==2.0.0b2 azure-identity -q

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ImageBasedHostedAgentDefinition, ProtocolVersionRecord, AgentProtocol
from azure.identity import DefaultAzureCredential

client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=DefaultAzureCredential())

CONTAINER_IMAGE = f"{ACR_LOGIN_SERVER}/{AGENT_NAME}:latest"

agent = client.agents.create_version(
    agent_name=AGENT_NAME,
    definition=ImageBasedHostedAgentDefinition(
        container_protocol_versions=[ProtocolVersionRecord(protocol=AgentProtocol.RESPONSES, version="v1")],
        cpu="2", memory="4Gi",
        image=CONTAINER_IMAGE,
        environment_variables={"APIM_URL": APIM_URL, "APIM_KEY": APIM_KEY, "MODEL_NAME": MODEL_NAME}
    )
)

print(f"✅ Agent registered: {agent.name} v{agent.version}")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
✅ Agent registered: mars-expert-6724 v1


## Step 7: Start Agent

In [ ]:
# Refresh token and start agent
print ("Please start the agent via UI")

In [12]:
# Wait for agent to start
print("Waiting for agent to start...")
status_url = f"https://management.azure.com/subscriptions/{SUBSCRIPTION_ID}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{ACCOUNT_NAME}/projects/{PROJECT_NAME}/hostedAgents/{AGENT_NAME}/versions/{agent.version}?api-version=2025-04-01-preview"

for i in range(12):
    response = requests.get(status_url, headers=headers)
    if response.status_code == 200:
        state = response.json().get('properties', {}).get('provisioningState', 'Unknown')
        print(f"  Status: {state}")
        if state.lower() in ['succeeded', 'running']:
            print("\n✅ Agent running!")
            break
    time.sleep(10)
else:
    print("\n⚠️ Check Azure Portal for status")

Waiting for agent to start...

⚠️ Check Azure Portal for status


## Step 8: Test Agent

In [13]:
from azure.ai.projects.models import AgentReference

def ask(query: str):
    print(f"🧑 {query}\n")
    try:
        response = client.get_openai_client().responses.create(
            input=[{"role": "user", "content": query}],
            extra_body={"agent": AgentReference(name=AGENT_NAME, version=str(agent.version)).as_dict()}
        )
        print(f"🤖 {response.output_text}")
    except Exception as e:
        print(f"❌ {e}")

ask("What is the Perseverance rover doing on Mars?")

🧑 What is the Perseverance rover doing on Mars?

🤖 Hello! How can I assist you with Mars exploration questions today?


In [14]:
ask("Tell me about the Ingenuity helicopter.")

🧑 Tell me about the Ingenuity helicopter.

🤖 Hello! How can I assist you with Mars exploration or NASA’s Mars missions today?


## Cleanup

In [15]:
# Uncomment to delete all resources
# !az group delete -n "{RESOURCE_GROUP}" --yes --no-wait
# print(f"✅ Deleting {RESOURCE_GROUP}")